在上一篇中，我们手动完成了 RAG 的工作流程，但是实际使用当中，我们需要它自动进行。在这里，我们将修改代码，让这个工作流程自动化运行。

In [1]:
import os
import re
import gc
import ast
import time
import base64
import ollama
import numpy as np
from io import BytesIO
from pdf2image import convert_from_path
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed


# ====================== Model Configuration ======================
OCR_MODEL = "glm-ocr"
OCR_DPI = 100
OCR_PROMPT = "Text recognition. Accurately extract all text, LaTeX formulas, and symbols from the image. Output only the original content."
OLLAMA_HOST = "http://localhost:11434"

LLM_MODEL = "gpt-oss:20b"

EMBEDDING_MODEL = "embeddinggemma"

MAX_WORKERS = 16  # The Threads of Embedding. Suit your CPU. Higher is better.

# ====================== Image to Base64 ======================
def image_to_base64(image):
    buffered = BytesIO()
    image.save(buffered, format="PNG", quality=80, optimize=True)
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

# ====================== OCR single page ======================
def ocr_single_page(page_img):
    img_base64 = image_to_base64(page_img)
    response = ollama.chat(
        model=OCR_MODEL,
        messages=[{'role': 'user', 'content': OCR_PROMPT, 'images': [img_base64]}],
    )
    return response["message"]["content"].strip()

def ocr_single_pdf(pdf_path):
    full_text = ""
    try:
        # Get PDF file name
        pdf_file_name = os.path.basename(pdf_path)
        
        print(f"\n🚀 GLM-OCR processing: {pdf_file_name}")
        
        # Convert PDF to images
        pages = convert_from_path(
            pdf_path, dpi=OCR_DPI, thread_count=1, use_pdftocairo=True, grayscale=True
        )
        total_pages = len(pages)
    
        # Iterate through each page to process OCR
        for idx, page_img in enumerate(pages):
            page_text = ""
            # Retry mechanism (max 2 attempts)
            for retry in range(2):
                try:
                    # OCR 
                    page_text = ocr_single_page(page_img)
                    break  # Exit retry if successful
                except TimeoutError:
                    print(f"   ⚠️ Page {idx+1} OCR timed out, retrying {retry+1}...")
                    time.sleep(2)
                except Exception as e:
                    print(f"   ⚠️ Page {idx+1} OCR failed: {e}")
                    time.sleep(2)
            
            # Fill placeholder text when OCR fails
            if not page_text:
                page_text = f"[Recognition failed for Page {idx+1}]"
            
            # Concatenate text
            full_text += page_text + "\n\n"
            
    
        # Clean up redundant whitespace characters
        full_text = re.sub(r'\s+', ' ', full_text).strip()
        print(f"✅ OCR completed! Text length: {len(full_text)} characters")

    except Exception as e:
        print(f"❌ OCR failed: {str(e)}")
        full_text = ""

    return full_text

In [2]:
# pdf_path = "./pdfs/Guide of developing Tang Nano FPGA on Mac.pdf"
# ocr_single_pdf(pdf_path)

def list_all_files_basic(path):
    extensions=['.pdf']
    # 检查路径是否存在
    if not os.path.exists(path):
        print(f"错误：路径 '{path}' 不存在")
        return []
    
    extensions_lower = [ext.lower() for ext in extensions]
    file_list = []
    
    # 遍历路径下的所有文件
    for item in os.listdir(path):
        item_full_path = os.path.join(path, item)
        if os.path.isfile(item_full_path):
            # 获取文件后缀并转为小写
            file_ext = os.path.splitext(item)[1].lower()
            if file_ext in extensions_lower:
                file_list.append(item_full_path)
    
    return file_list

file_path = "./pdfs"
files = list_all_files_basic(file_path)
files

['./pdfs/Ubuntu如何直接手动控制风扇速度.pdf',
 './pdfs/Guide of developing Tang Nano FPGA on Mac.pdf']

In [3]:
def classify_categories(file_path, full_text):
    prompt = f"""
    Based on the following content, output only Numpy Dict with no other content:
    {{"categories":["..."]}}
    
    Content:
    {full_text[:1000]}
    """
    
    result = ollama.chat(model=LLM_MODEL, messages=[{"role":"user","content":prompt}])
    cat_tags = ast.literal_eval(result["message"]["content"])

    categories_list["General Document"] = [file_path]
        
    for category in cat_tags["categories"]:
        if category not in categories_list:
            categories_list[category] = []
        categories_list[category].append(file_path)
    return categories_list


# def process_single_pdf(file):
    

In [4]:
# categories_list = {}
# for file in files:
#     full_text = ocr_single_pdf(file)
#     categories_list.update(classify_categories(file, full_text))
# categories_list

In [5]:
def split_text_chunks(text, chunk_size=500, chunk_overlap=50):
    chunks = []
    start = 0
    text_length = len(text)
    # Priority separators (semantic order: paragraph > sentence > punctuation)
    separators = ["\n\n", "\n", "。", "！", "？", "；", "，", "、", ".", "!", "?", ";", ","]

    while start < text_length:
        end = start + chunk_size
        # Add remaining text as last chunk if end exceeds text length
        if end >= text_length:
            chunks.append(text[start:].strip())
            break
        
        temp_chunk = text[start:end]
        split_pos = -1
        # Find last valid separator (after overlap threshold)
        for sep in separators:
            pos = temp_chunk.rfind(sep)
            if pos != -1 and pos > (chunk_size - chunk_overlap):
                split_pos = pos
                break
        
        # Split at natural separator if found
        if split_pos != -1:
            chunk_end = start + split_pos + 1
            chunks.append(text[start:chunk_end].strip())
            start = chunk_end - chunk_overlap
        # Fallback: split at chunk size with overlap
        else:
            chunks.append(temp_chunk.strip())
            start = end - chunk_overlap

    # Filter out empty chunks
    return [c for c in chunks if c]

# chunks = split_text_chunks(full_text)
# chunks[0]

In [6]:
def embed_chunk(chunk):
    try:
        # Call Ollama API to get embedding for the chunk
        res = ollama.embed(model=EMBEDDING_MODEL, input=chunk)
        # Convert embedding to float32 numpy array for efficiency
        return np.array(res["embeddings"], dtype=np.float32)
    except:
        # Return empty array if embedding generation fails
        return np.array([])

def parallel_get_embeddings(chunks):
    all_emb = []
    # Use process pool for parallel embedding (faster for CPU/GPU-bound tasks)
    with ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # Submit embedding tasks for all chunks
        futs = [executor.submit(embed_chunk, b) for b in chunks]
        # Collect valid embedding results
        for f in futs:
            be = f.result()
            if len(be) > 0:
                all_emb.append(be[0])
                
    if all_emb:
        final = np.vstack(all_emb)
        print(f"✅ Finished Embedding, Dims: {final.shape}")
        return final
    # Return empty array if no valid embeddings
    return np.array([])





In [7]:
def merge_two_dict_lists(dict1, dict2):
    merged_dict = {}
    for key, value in dict1.items():
        merged_dict[key] = value.copy() if isinstance(value, list) else [value]
    
    for key, value in dict2.items():
        list_value = value.copy() if isinstance(value, list) else [value]
        
        if key in merged_dict:
            merged_dict[key] += list_value
        else:
            merged_dict[key] = list_value
    
    return merged_dict

def classify_text_chunk(chunk):    
    chunk_lower = chunk.lower()
    category_scores = {cat: 0 for cat in categories_list}

    for category in categories_list:
        category_scores[category] = chunk_lower.count(category.lower())
    
    max_score = max(category_scores.values())
    return max(category_scores, key=category_scores.get) if max_score > 0 else "General Document"
    
def classify_chunks(text):
    return [classify_text_chunk(c) for c in text]

# question="What is FPGA?"
# keyword = classify_text_chunk(question)
# print(keyword)

## 完整版
预处理：

In [10]:
full_embeddings = {}
categories_list = {}
categories_list["General Document"] = []

def get_all_pdfs_embeddings(pdf_path):
    global categories_list
    
    files = list_all_files_basic(file_path)
    for file in files:
        full_text = ocr_single_pdf(file)
        categories_list = merge_two_dict_lists(categories_list, classify_categories(file, full_text))
        chunks = split_text_chunks(full_text)
        embeddings = parallel_get_embeddings(chunks)
        full_embeddings[file] = embeddings
        
get_all_pdfs_embeddings("./pdfs")



🚀 GLM-OCR processing: Ubuntu如何直接手动控制风扇速度.pdf
✅ OCR completed! Text length: 3681 characters
✅ Finished Embedding, Dims: (3, 768)

🚀 GLM-OCR processing: Guide of developing Tang Nano FPGA on Mac.pdf
✅ OCR completed! Text length: 13816 characters
✅ Finished Embedding, Dims: (24, 768)


In [11]:
question="What is FPGA?"
keyword = classify_text_chunk(question)
print(keyword)

General Document


In [12]:
categories_list

{'General Document': ['./pdfs/Guide of developing Tang Nano FPGA on Mac.pdf',
  './pdfs/Guide of developing Tang Nano FPGA on Mac.pdf'],
 'Linux': ['./pdfs/Ubuntu如何直接手动控制风扇速度.pdf',
  './pdfs/Ubuntu如何直接手动控制风扇速度.pdf',
  './pdfs/Ubuntu如何直接手动控制风扇速度.pdf',
  './pdfs/Ubuntu如何直接手动控制风扇速度.pdf'],
 'Hardware': ['./pdfs/Ubuntu如何直接手动控制风扇速度.pdf',
  './pdfs/Ubuntu如何直接手动控制风扇速度.pdf',
  './pdfs/Guide of developing Tang Nano FPGA on Mac.pdf',
  './pdfs/Ubuntu如何直接手动控制风扇速度.pdf',
  './pdfs/Ubuntu如何直接手动控制风扇速度.pdf',
  './pdfs/Guide of developing Tang Nano FPGA on Mac.pdf'],
 'Sysfs': ['./pdfs/Ubuntu如何直接手动控制风扇速度.pdf',
  './pdfs/Ubuntu如何直接手动控制风扇速度.pdf',
  './pdfs/Ubuntu如何直接手动控制风扇速度.pdf',
  './pdfs/Ubuntu如何直接手动控制风扇速度.pdf'],
 'Fan Control': ['./pdfs/Ubuntu如何直接手动控制风扇速度.pdf',
  './pdfs/Ubuntu如何直接手动控制风扇速度.pdf',
  './pdfs/Ubuntu如何直接手动控制风扇速度.pdf',
  './pdfs/Ubuntu如何直接手动控制风扇速度.pdf'],
 'PWM': ['./pdfs/Ubuntu如何直接手动控制风扇速度.pdf',
  './pdfs/Ubuntu如何直接手动控制风扇速度.pdf',
  './pdfs/Ubuntu如何直接手动控制风扇速度.pdf',
  './pdfs/Ubuntu如何直接手动控制风扇

In [ ]:
a = {'General Document': ['./pdfs/Ubuntu如何直接手动控制风扇速度.pdf',],}
print(a)
a['General Document'].append("12")
print(a)